### DATA PREPROGRESS

In [19]:
import numpy as np
import pandas as pd

def one_hot_encoding(arr):
    s = set()
    _sd = dict()
    for e in arr:
        s.add(e)

    sl = list(s)
    for i in range(len(sl)):
        encode = np.zeros(len(sl))
        encode[i] = 1
        _sd[sl[i]] = encode

    encoded_data = []
    for k in range(len(arr)):
        encoded_data.append(_sd[arr[k]])

    return np.array(encoded_data)


def data_preprocess(filepath):
    df = pd.read_excel(filepath)
    data = df.to_numpy()

    ip = data[:, 2]

    encoded_ip = one_hot_encoding(ip)

    # print(encoded_ip)

    data[:, 2] = encoded_ip.tolist()

    # print(data)
    expanded_data = []
    for row in data:
        expanded_data.append(np.concatenate([row[:2], row[2], row[3:]]).tolist())

    expanded_data = np.array(expanded_data)

    print(expanded_data[0])

    for i in range(len(expanded_data)):
        for j in range(len(expanded_data[i])):
            expanded_data[i][j] = float(expanded_data[i][j])

    return expanded_data[:70]


filename = "outputv2_5_wait_5s_2"

# Data preprocessing
file_path = f"D:\\model_fit\\training\\training_data\\{filename}.xlsx"
dataset = data_preprocess(file_path)



[3.63330e+00 2.12984e+05 1.00000e+00 0.00000e+00 0.00000e+00 3.84615e+01
 3.79310e+01 3.02548e+01]


### MODEL COMPILE & FIT

In [20]:
### Data preprocessing using MinMaxScaler
# create MinMaxScaler Object

from sklearn.preprocessing import MinMaxScaler

x_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()


# scaled_training_x = training_x[:, 0].reshape(-1, 1)
# scaled_training_y = training_y[:].reshape(-1, 1)

# training_x[:, 0] = scaled_training_x[:, 0]
# training_y = scaled_training_y

# Define the split points based on time
train_size = int(len(dataset) * 0.7)
val_size = int(len(dataset) * 0.2)

# Split the data
train = dataset[:train_size]
val = dataset[train_size : train_size + val_size]

# test data from other file
test = dataset[train_size + val_size :]

# x_train_first_column = x_train[:, 0].reshape(-1, 1)
# scaled_x_train_first_column = x_scaler.fit_transform(x_train_first_column)
# x_train[:, 0] = scaled_x_train_first_column.flatten()
# y_train_scaled = y_scaler.fit_transform(y_train.reshape(-1, 1))  # reshap to 2D

# make the same sacling on valid data set and test data set
# x_val_scaled = x_scaler.transform(x_val)
# x_val_first_column = x_val[:, 0].reshape(-1, 1)
# scaled_x_val_first_column = x_scaler.fit_transform(x_val_first_column)
# x_val[:, 0] = scaled_x_val_first_column.flatten()
# y_val_scaled = y_scaler.transform(y_val.reshape(-1, 1))  # reshap to 2D


# x_test_first_column = x_test[:, 0].reshape(-1, 1)
# scaled_x_test_first_column = x_scaler.fit_transform(x_test_first_column)
# x_test[:, 0] = scaled_x_test_first_column.flatten()
# y_test_scaled = y_scaler.transform(y_test.reshape(-1, 1))  # reshap to 2D

### NORMALIZATION

In [21]:
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Input, Bidirectional, Flatten, TimeDistributed, Conv1D, MaxPooling1D, Conv2D
from keras.models import Model
from keras.callbacks import Callback, EarlyStopping
from keras.optimizers import Adam


def create_sequences(dataset, time_steps):
    dataX, dataY = list(), list()
    for i in range(len(dataset)):
        endx = i + time_steps

        if endx > len(dataset) - 1:
            break

        dataset_x = dataset[i:endx, 1]
        dataset_y = dataset[endx, 0]

        dataX.append(dataset_x)
        dataY.append(dataset_y)

    return np.array(dataX), np.array(dataY)

# Create sequences for each set
time_steps = 1


# x_test = training_x[train_size:]
# y_test = one_hot_encoding[train_size:]

# Create sequences for each set
X_train, y_train = create_sequences(train, time_steps)
X_val, y_val = create_sequences(val, time_steps)
X_test, y_test = create_sequences(test, time_steps)


# Initialize LossHistory with validation data
class LossHistory(Callback):
    def __init__(self, x_train, y_train, x_val, y_val):
        super().__init__()
        self.train_data = (x_train, y_train)
        self.validation_data = (x_val, y_val)
        self.losses = []
        self.val_losses = []
        self.train_errors = []
        self.val_errors = []

    def on_epoch_end(self, epoch, logs=None):
        self.losses.append(logs['loss'])
        self.val_losses.append(logs.get('val_loss'))
        self.train_errors.append(logs.get('mean_absolute_error'))
        self.val_errors.append(logs.get('val_mean_absolute_error'))


print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(48, 1)
(13, 1)
(6, 1)


### FITTING

In [22]:
from keras.callbacks import TensorBoard
import os
import datetime


# log
log_dir = os.path.join("logs", "fit", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

def add_hidden_layer(
    model,
    units,
    layers,
    activation=None,
):
    if activation:
        if layers > 1:
            for layer in range(layers - 1):
                model.add(
                    LSTM(units=units, activation=activation, return_sequences=True)
                )
        model.add(LSTM(units=units, activation=activation, return_sequences=False))
    else:
        if layers > 1:
            for layer in range(layers - 1):
                model.add(LSTM(units=units, return_sequences=True))
        model.add(LSTM(units=units, return_sequences=False))


# loop params

LSTM_units = 50

Stop_patience = 20

Batch_size = 64

Learning_rate = 1e-3

Epochs = 100

Layers = 0

Activation = "relu"


# loss_function = 'mean_absolute_error'  # Assuming you are doing regression

# loss_function = 'categorical_crossentropy'

# loss_function = 'sparse_categorical_crossentropy'

loss_function = "mean_squared_error"


# Initialize LossHistory with validation data

history = LossHistory(
    X_train, y_train, X_val, y_val
)

def model_fit():

    print("epochs:", Epochs)

    print("batch:", Batch_size)

    print("units:", LSTM_units)

    print(X_train.shape)

    # Build LSTM model
    model = Sequential()
    model.add(Input(shape=(time_steps, 1)))

    # Adding LSTM layer
    model.add(LSTM(units=LSTM_units, activation=Activation, return_sequences=False))
    model.add(Dense(1, activation="linear"))

    # Compile model
    model.compile(
        optimizer=Adam(learning_rate=Learning_rate),
        loss=loss_function,
        metrics=["mean_absolute_error"],
    )

    # Define early stopping

    early_stopping = EarlyStopping(
        monitor="val_loss", patience=Stop_patience, restore_best_weights=True
    )

    # early_stopping = EarlyStopping(monitor='val_mean_absolute_error', patience=Stop_patience, restore_best_weights=True)

    # Train the model with early stopping
    print(X_train.shape)

    model.fit(
        X_train,
        y_train,
        epochs=Epochs,
        batch_size=Batch_size,
        validation_data=(X_val, y_val),
        callbacks=[history, tensorboard_callback],
        # callbacks=[history, early_stopping, tensorboard_callback],
    )

    prediction_results = model.predict(X_test)

    return prediction_results


prediction_results = model_fit()

epochs: 100
batch: 64
units: 50
(48, 1)
(48, 1)
Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 5252806656.0000 - mean_absolute_error: 63392.4805 - val_loss: 4158182400.0000 - val_mean_absolute_error: 55482.8086
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step - loss: 5120478720.0000 - mean_absolute_error: 62588.9531 - val_loss: 4052682240.0000 - val_mean_absolute_error: 54774.4805
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step - loss: 4990566400.0000 - mean_absolute_error: 61789.9258 - val_loss: 3949154048.0000 - val_mean_absolute_error: 54070.3672
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step - loss: 4863082496.0000 - mean_absolute_error: 60995.6680 - val_loss: 3847607808.0000 - val_mean_absolute_error: 53370.7109
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step - loss: 4738038272.0000 - mean_absolute_error: 60206.4258 - val_loss: 3748050432.0000 - val_mean_absolute_error: 52675.7422
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step - loss: 4615444480.0000 - mea

In [23]:
_abs = np.abs(y_test - prediction_results)
mae = np.mean(_abs)
print("MAE:", mae)
for i in range(len(prediction_results)):
    print(f"pred: {prediction_results[i]} | real: {y_test[i]}")
avg = np.mean(y_test)
print(avg)

MAE: 9651.671505135088
pred: [-413.13107] | real: 2.0314
pred: [-6236.984] | real: 13.053
pred: [-22776.855] | real: 5.2372
pred: [-11975.169] | real: 1.9604
pred: [-6060.394] | real: 4.2326
pred: [-10408.774] | real: 12.2066
6.453533333333334


### SAVE MODEL

In [24]:
# Save the model
    # model.save("/content/drive/MyDrive/LSTM/predict_model_2_0v.keras")